In [ ]:
!pip install openai==0.28
!pip install tiktoken

In [ ]:
!pip install openai --upgrade

In [ ]:
import pandas as pd
import json
from openai import OpenAI
import time
import os

In [ ]:
client = OpenAI(
    api_key=os.environ.get("OPENAI_API_KEY")
)

In [ ]:
def batch_gpt(message_prompt):
    responses = []
    response = client.chat.completions.create(
        model="gpt-4-0125-preview",
        messages=[{"role": "user", "content": message_prompt}]
    )
    r = response.choices[0].message.content
    responses.append(r)

    return r, responses

In [ ]:
def get_mess():
  mess = """
You need to find the existing discourse relations between 2 elementary discourse units. Discourse relations between 2 discourse units can be explicit or implicit.
I will give you a text and 2 elementary discourse units extracted from that text and then i will provide you with all the 12 possible discourse relation types from which you should choose one. In the context of the example discourse units, discourse_unit_1 comes before discourse_unit_2. From the give relaions_types you should  determine the type of existing discourse relation between 2 given example elementary discourse units.
Return the result in json format  with no addition explanation:
example:
{"discourse_relation" : "found_discourse_relation"}
      """

  return mess

In [ ]:
dataset = pd.read_csv('/local_path/eng.dep.covdtb_test.rels.csv', sep='\t')
df = pd.DataFrame(columns=['EDU_1', 'EDU_2', 'Real discourse relation', 'ChatGPT discourse relation'])
unprocessed_data = pd.DataFrame(columns=['unit1_sent', 'unit2_sent', 'dir', 'label'])

In [ ]:
dataset.head()

,doc,unit1_toks,unit2_toks,unit1_txt,unit2_txt,s1_toks,s2_toks,unit1_sent,unit2_sent,dir,orig_label,label
0,00c862a6402381a8541bcffe843696e059095300,1-18,19-22,"In this study , we investigated turkey reoviru...",aged 15 days .,1-22,1-22,"In this study , we investigated turkey reoviru...","In this study , we investigated turkey reoviru...",1<2,elaboration,elaboration
1,00c862a6402381a8541bcffe843696e059095300,1-18,23-32,"In this study , we investigated turkey reoviru...",RT - PCR for TReoV detected 3.3 % positive sam...,1-22,23-41,"In this study , we investigated turkey reoviru...",RT - PCR for TReoV detected 3.3 % positive sam...,1<2,findings,findings
2,00c862a6402381a8541bcffe843696e059095300,23-32,33-41,RT - PCR for TReoV detected 3.3 % positive sam...,and TReoV was successfully isolated in Vero ce...,23-41,23-41,RT - PCR for TReoV detected 3.3 % positive sam...,RT - PCR for TReoV detected 3.3 % positive sam...,1<2,joint,joint
3,00c862a6402381a8541bcffe843696e059095300,23-32,42-58,RT - PCR for TReoV detected 3.3 % positive sam...,Histological analysis of positive bursa of Fab...,23-41,42-58,RT - PCR for TReoV detected 3.3 % positive sam...,Histological analysis of positive bursa of Fab...,1<2,elaboration,elaboration
4,00c862a6402381a8541bcffe843696e059095300,42-58,59-76,Histological analysis of positive bursa of Fab...,"The number of CD8 ? , CD4 ? and IgM ? cells wa...",42-58,59-76,Histological analysis of positive bursa of Fab...,"The number of CD8 ? , CD4 ? and IgM ? cells wa...",1<2,elaboration,elaboration


In [ ]:
dataset.shape

(2586, 12)

In [ ]:
data_with_texts = pd.read_csv('/local_path/eng_dep_covdtb_texts_test.csv')
data_with_texts.head()

,Document ID,Document Abstract
0,00c862a6402381a8541bcffe843696e059095300,"In this study , we investigated turkey reoviru..."
1,0146a0b36f80ebcb868c7e16fb60c1a9d2afbf83,This is an extensive review on epiphytic plant...
2,0232f18e7bbf3192e9e34da1c0abe2a20b2ba1e4,"Herein , the LASSBio Chemical Library is prese..."
3,09bdbea1700a3826acc33484ed51e2fa8c23f047,Data - centric models of COVID-19 have been tr...
4,0b362bc7e9278b56c2740ab7bc10e91c036f835a,We study seasonal epidemic spreading in a susc...


In [ ]:
message_part1 = get_mess()
message_part2 = {
      'elaboration',
      'findings',
      'joint',
      'attribution',
      'manner-means',
      'enablement',
      'cause-result',
      'background',
      'comparison',
      'textual-organization',
      'temporal',
      'condition'
  }

for i in range(0, len(dataset)):
  print()
  doc_id = dataset['doc'][i]
  sentence_1 = dataset['unit1_txt'][i]
  sentence_2 = dataset['unit2_txt'][i] 
  real_label = dataset['label'][i]

  row = data_with_texts[data_with_texts['Document ID'] == doc_id]
  document_text = row['Document Abstract'].values[0]

  message_part3 = {
      "text" : document_text
  }

  message_part4 = {
        "discourse_unit_1" : sentence_1,
        "discourse_unit_2" : sentence_2
  }

  prompt_message = message_part1 + '\n' + 'relation_types = ' + str(message_part2) + '\n' + '\nexample_text = ' + str(message_part3) + '\n' + '\nexample_discourse_units = ' + str(message_part4)
  print(str(i) + " " + str(message_part4))
  try:
    result, response = batch_gpt(prompt_message)
    result = result.replace('json', '')
    result = result.replace('`', '')
    result = result.strip()
    print(result)
    json_result = json.loads(result)
    gpt_relation = json_result['discourse_relation']
  except Exception as e:
    print("ERROR")
    print(e)
    unprocessed_data.loc[len(unprocessed_data)] = {'unit1_txt': dataset['unit1_txt'][i], 'unit2_txt': dataset['unit2_txt'][i], 'dir': dataset['dir'][i], 'label': real_label}
  else:
    new_row = {'EDU_1': sentence_1, 'EDU_2': sentence_2, 'Real discourse relation': real_label, 'ChatGPT discourse relation': gpt_relation}
    df.loc[len(df)] = new_row
    if i % 300 == 0:
      time.sleep(10)


In [ ]:
df.tail()

,EDU_1,EDU_2,Real discourse relation,ChatGPT discourse relation
568,The review highlights on the ethical norms,which needs to be adhered to,elaboration,elaboration
569,which needs to be adhered to,while receiving the human body of a donor and ...,temporal,manner-means
570,The review highlights on the ethical norms,It reflects on ideal ethical behaviour in the ...,elaboration,elaboration
571,The present review is an attempt,The intended purpose of this article is to sup...,enablement,elaboration
572,The intended purpose of this article is to sup...,used in anatomical dissection .,elaboration,elaboration


In [ ]:
unprocessed_data.to_csv('eng_dep_covdtb_test_unprocessed_prompt_messagges.csv', index=False)
df.to_csv('gpt_4_eng_dep_covdtb_test_results.csv', index=False)

In [ ]:
df['ChatGPT discourse relation'].unique()

array(['elaboration', 'joint', 'temporal', 'no clear relation',
       'background', 'purpose', 'cause-result', 'manner-means',
       'enablement', 'contrast', 'comparison', 'attribution', 'condition',
       'textual-organization', 'concession'], dtype=object)

In [ ]:
df['Real discourse relation'].unique()

array(['elaboration', 'findings', 'joint', 'attribution', 'manner-means',
       'enablement', 'cause-result', 'background', 'comparison',
       'textual-organization', 'temporal', 'condition'], dtype=object)

In [ ]:
not_included = ['no clear relation', 'concession', 'purpose', 'condition', 'contrast']
br = 0
for i in range(len(df)):
  if df.iloc[i, 3] in not_included:
    br += 1
print(br)

40
